In [180]:
import pickle
import torch
import pandas as pd
from torch_geometric.data import Data,DataLoader
from functions import *
from pytorch_util import *
from torch.optim import Adam
from torch.nn.utils import clip_grad_value_

In [2]:
with open('../Data/train_data3.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('../Data/val_data3.pickle', 'rb') as handle:
    val_data = pickle.load(handle)
with open('../Data/test_data3.pickle', 'rb') as handle:
    test_data = pickle.load(handle)

In [144]:
### parameters ###
batch_size = 32
dim = 64
edge_dim = 12
epochs = 5
clip = 2

### load dataset

In [134]:
train_list = [Data(**d) for d in train_data]
train_dl = DataLoader(train_list,batch_size,shuffle=True)

In [135]:
val_list = [Data(**d) for d in val_data]
valid_dl = DataLoader(val_list,batch_size,shuffle=False)

In [172]:
test_list = [Data(**d) for d in test_data]
test_dl = DataLoader(test_list,batch_size,shuffle=False)

### build model and set up training

In [153]:
model = Net(dim=dim,edge_dim=edge_dim).to('cuda:0')

In [154]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=1e-4)

In [155]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))

time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:1.7602821293082147, val_loss:1.5587950002433908
epoch:1, train_loss:1.4830898141337683, val_loss:1.418043826889788
epoch:2, train_loss:1.3485713588302954, val_loss:1.275693492247508
epoch:3, train_loss:1.1774547664771409, val_loss:1.1019265121883817
epoch:4, train_loss:0.9942853674258497, val_loss:0.9160097324185901
Training completed in 94.866375207901s


In [156]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))

time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.8484271750038053, val_loss:0.7908172900350685
epoch:1, train_loss:0.7262328077409166, val_loss:0.7192818637077625
epoch:2, train_loss:0.6805284209702731, val_loss:0.6647164438270096
epoch:3, train_loss:0.6496724044835126, val_loss:0.6365684647208605
epoch:4, train_loss:0.6261484850526783, val_loss:0.6232077294053175
Training completed in 94.65523791313171s


### make submissions

In [174]:
model.eval()
yhat_list = []
with torch.no_grad():
    for data in test_dl:
        data = data.to('cuda:0')
        yhat_list.append(model(data,False))

yhat = torch.cat(yhat_list).cpu().detach().numpy()
submission = pd.read_csv('../Data/sample_submission.csv')
submission['scalar_coupling_constant'] = yhat
submission.to_csv('../Submission/sub0.csv',index=False)